## cisTopic results on scATAC-seq dataset (GSE96769)

## Кратко о методе LDA

__cisTopic__ основывается на $LDA$ _(Latent Dirichlet allocation)_***, байесовском методе, который используется для _unsupervised_ классификации текстов по частоте встречаемости слов.  

На вход подаётся матрица $D * W$ _(документы / частоты встречаемости слов в них)_, a $LDA$ факторизует эту матрицу на две:
+ $D*T$ _(документы / присваемые темы)_ - Матрица вероятности принадлежности $i$ документа к $j$ теме (топику).
+ $T*W$  _(темы / слова)_  - Матрица вероятности принадлежности $j$ слова к $i$ теме (топику).


В отличие от наивого байесовского классификатора, для $LDA$ темы не выбираются из заранее определённого дискретного множества категорий, но мы предполагаем, что документы в обучающей выборке содержат заданное нами число тем. При этом $LDA$ классификатор не является наивым, потому что при присваивании слова к одному из топиков, он смотрит не только на то, как часто данное слово встречалось по всем документам, но и к каким топикам были присвоены другие слова в этом документе.

Если более строго, целевая функция $LDA$:
![objective_function.png](./cisTopic_results/objective_function.png)
где:  
$\alpha$ - распределение заданных тем,  
$\beta$ - распределение слов в темах,  
$\beta_с$ - распределение слов, соответсвующее теме $с$  
$D \in \mathcal{D}$ - документ из множества документов обучающей выборки,  
$w$ - распределение слов в документе

Т.о. при обучении каждая тема формируется так, чтобы она могла наилучшим образом отделять группу документов со сходим распределением слов от общего множества документов. Другими словами можно сказать, что при обучении максимизуется _общее правдоподобие_:
![likelihood.png](./cisTopic_results/likelihood.png)

Авторы же __cisTopic__ принимают клетки за документы, а модули открытых регионов хроматина за темы. Таким образом, каждый регион в каждой клетке назначается модулю (топику), основываясь на том, к какому топику этот регион был назначен в других клетках и к каким топикам были назначены другие регионы в этой клетке. После обучения эти импутации используются для оценки вероятности принадлежности региона к цис-регуляторному топику _(region-topic distribution)_ и вкладе каждого топика в клетке _(topic-cell distribution)_.

*** $LDA$ c [сэмплированием по Гиббсу][1], которое облегчает задачу кластеризации, так как позволяет перейти от выраженных совместных распределений к условным вероятностям.

Подробнее o LDA:  
+ https://habr.com/ru/post/417167/,  
+ https://habr.com/ru/company/surfingbird/blog/228249/

[1]:(https://ru.wikipedia.org/wiki/%D0%A1%D0%B5%D0%BC%D0%BF%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D0%BE_%D0%93%D0%B8%D0%B1%D0%B1%D1%81%D1%83)

## Результаты

Обучение проводилось на двух версиях датасета GSE96769:
+ Полный версия датасета:  
  __2778 клеток__ и __597450 регионов__ 
+ Уменьшенная версия, содержащая только регионы +/- 10 kbp около генов:  
  __2778 клеток__ и __149850 регионов__

### Полный датасет

![cisTopic_model_selection.png](./cisTopic_results/GSE96769_scATAC_all_peaks/cisTopic_model_selection.png)
__Fig. 1.__ cisTopic model selection by picking model with the highest log likelihood ($\log(P(D|T))$)  

![likelihood_stabilisation_across_models.png](./cisTopic_results/GSE96769_scATAC_all_peaks/likelihood_stabilisation_across_models.png)
__Fig. 2.__ Likelihood stabilization across models

![Regions_num_per_topic.png](./cisTopic_results/GSE96769_scATAC_all_peaks/Regions_num_per_topic.png)
__Fig. 3.__ Number of regions selected for each topic

![UMAP_2d_clustering.png](./cisTopic_results/GSE96769_scATAC_all_peaks/UMAP_2d_clustering.png)
__Fig. 4.__ UMAP 2D clusterization on learned _topic-cell distribution_ (15 components)

![UMAP_3d_clustering.png](./cisTopic_results/GSE96769_scATAC_all_peaks/UMAP_3d_clustering.png)
__Fig. 5.__ UMAP 3D clusterization on learned _topic-cell distribution_ (15 components)

![Heatmap_topic_contrib_per_cell.png](./cisTopic_results/GSE96769_scATAC_all_peaks/Heatmap_topic_contrib_per_cell.png)
__Fig. 6.__ Heatmap of topics contribution per cell with group cells and topics clusterisation

![UMAP_2d_topic_1_9.png](./cisTopic_results/GSE96769_scATAC_all_peaks/UMAP_2d_topic_1_9.png)
__Fig. 7.__ UMAP 2D clusterisation of _topic-region distribution_ (15 components)

![Topic_annotation_to_genes.png](./cisTopic_results/GSE96769_scATAC_all_peaks/Topic_annotation_to_genes.png)
__Fig. 8.__ Topics annotation to the _hg19_ regions

![Topics_GO_enrichment.png](./cisTopic_results/GSE96769_scATAC_all_peaks/Topics_GO_enrichment.png)
__Fig. 9.__ GO enrichment of genes which their body and distal elements intersect with topic regions

__Получение эухроматиновых "подписей" для типов клеток__

Т.к. cisTopics требует каких-то подписей для клеток, было сделано следующее:
+ Из исходной матрице _(клетки / регионы)_ делалась бинаризированная
+ По данной матрице для кажого региона считалось среднее арифметическое его встречаемости в каждом типа клеток
+ Для каждого типа клеток выбирались только те открытые регионы, которые встречались > 30% ***
+ Аналогично выбирались регионы, которые встречались только в данном типе клеток

*** Альтернативно пробовал 50% и 40%, однако получал почти нулевую матрицу (доля ненулевых элементов 0.015 и 0.021 соответвенно)

![bulk_signatures_heatmap.png](./cisTopic_results/GSE96769_scATAC_all_peaks/bulk_signatures_heatmap.png)
__Fig. 10.__ Heatmap of noramlised AUC scores of topics regions and _cell-type-specific signatures_, __containing overlaping regions__

![unique_bulk_signatures_heatmap.png](./cisTopic_results/GSE96769_scATAC_all_peaks/unique_bulk_signatures_heatmap.png)
__Fig. 11.__ Heatmap of noramlised AUC scores of topics regions and _cell-type-specific signatures_ (fully processed, without overlaping regions)

![unique_bulk_signatures_clusterization.png](./cisTopic_results/GSE96769_scATAC_all_peaks/unique_bulk_signatures_clusterization.png)
__Fig. 12.__ Noramlised AUC scores of topics regions and _cell-type-specific signatures_ (without overlaping regions) on clusterisaton of _cell-topic distribution_

## Уменьшенный датасет - регионы +/- 10 kbp около генов

![cisTopic_model_selection.png](./cisTopic_results/GSE96769_scATAC_10kbp_near_genes/cisTopic_model_selection.png)
__Fig. 1.__ cisTopic model selection by picking model with the highest log likelihood ($\log(P(D|T))$)

![likelihood_stabilisation_across_models.png](./cisTopic_results/GSE96769_scATAC_10kbp_near_genes/likelihood_stabilisation_across_models.png)
__Fig. 2.__ Likelihood stabilization across models

![Peaks_per_topics.png](./cisTopic_results/GSE96769_scATAC_10kbp_near_genes/Peaks_per_topics.png)
__Fig. 3.__ Number of regions selected for each topic

![UMAP_2d_clustering.png](./cisTopic_results/GSE96769_scATAC_10kbp_near_genes/UMAP_2d_clustering.png)
__Fig. 4.__ UMAP 2D clusterization on learned _topic-cell distribution_ (30 components)

![UMAP_3d_clustering.png](./cisTopic_results/GSE96769_scATAC_10kbp_near_genes/UMAP_3d_clustering.png)
__Fig. 5.__ UMAP 3D clusterization on learned _topic-cell distribution_ (30 components)

![heatmap_topic_contribution_per_cell.png](./cisTopic_results/GSE96769_scATAC_10kbp_near_genes/heatmap_topic_contribution_per_cell.png)
__Fig. 6.__ Heatmap of topics contribution per cell with group cells and topics clusterisation

![Topic_annotation_to_genes.png](./cisTopic_results/GSE96769_scATAC_10kbp_near_genes/Topic_annotation_to_genes.png)
__Fig. 8.__ Topics annotation to the _hg19_ regions

![Topics_GO_enrichment.png](./cisTopic_results/GSE96769_scATAC_10kbp_near_genes/Topics_GO_enrichment.png)
__Fig. 9.__ GO enrichment of genes which their body and distal elements intersect with topic regions

![unique_bulk_signatures_heatmap.png](./cisTopic_results/GSE96769_scATAC_10kbp_near_genes/unique_bulk_signatures_heatmap.png)
__Fig. 11.__ Heatmap of noramlised AUC scores of topics regions and _cell-type-specific signatures_ (fully processed, without overlaping regions)

![unique_bulk_signatures_clusterization.png](./cisTopic_results/GSE96769_scATAC_10kbp_near_genes/unique_bulk_signatures_clusterization.png)
__Fig. 12.__ Noramlised AUC scores of topics regions and _cell-type-specific signatures_ (without overlaping regions) on clusterisaton of _cell-topic distribution_